<a href="https://colab.research.google.com/github/moh-a-abde/moh-a-abde/blob/main/SVM_nuScenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# uncomment and run to get nuscenes dataset
#!mkdir -p /data/sets/nuscenes  # Make the directory to store the nuScenes dataset in.

#!wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.

#!tar -xf v1.0-mini.tgz -C /data/sets/nuscenes  # Uncompress the nuScenes mini split.

!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.

In [ ]:
#import libraries
import time
import os

import numpy as np
from numpy.random import randn
import pandas as pd

import math
from math import sqrt

import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns


import sklearn.preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, multilabel_confusion_matrix
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, balanced_accuracy_score
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##nuSCenes
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.geometry_utils import transform_matrix
from nuscenes.eval.detection.config import config_factory
from nuscenes.eval.detection.data_classes import DetectionBox
from nuscenes.eval.detection.evaluate import NuScenesEval
from sklearn.metrics import classification_report, accuracy_score

# Load the nuScenes dataset
nusc = NuScenes(version='v1.0-mini', dataroot='/data/sets/nuscenes', verbose=True)

# Initialize lists for features and labels
features = []
labels = []

# Iterate through samples and extract relevant data
for sample in nusc.sample:
    for ann_token in sample['anns']:
        ann = nusc.get('sample_annotation', ann_token)

        # Extract relevant features (e.g., position, dimensions)
        x, y, z = ann['translation']
        length, width, height = ann['size']

        # Append features and corresponding label
        features.append([x, y, z, length, width, height])
        labels.append(ann['category_name'])

# Convert to numpy arrays
features = np.array(features)
labels = np.array(labels)

AssertionError: Database version not found: /data/sets/nuscenes/v1.0-mini

In [ ]:
# Extract unique labels
unique_labels = np.unique(labels)

# Print the unique labels
print(unique_labels)

['human.pedestrian.adult' 'human.pedestrian.child'
 'human.pedestrian.construction_worker'
 'human.pedestrian.personal_mobility' 'human.pedestrian.police_officer'
 'movable_object.barrier' 'movable_object.debris'
 'movable_object.pushable_pullable' 'movable_object.trafficcone'
 'static_object.bicycle_rack' 'vehicle.bicycle' 'vehicle.bus.bendy'
 'vehicle.bus.rigid' 'vehicle.car' 'vehicle.construction'
 'vehicle.motorcycle' 'vehicle.trailer' 'vehicle.truck']


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training (60%) and temporary (40%) sets
X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size=0.4, random_state=42)

# Split the temporary set into validation (50% of 40% = 20%) and test (50% of 40% = 20%) sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Initialize the SVM model with specified parameters
svm_model = SVC(kernel='rbf', C=10, gamma=0.01, random_state=42)

# Train the SVM model
svm_model.fit(X_train, y_train)

# Validate the model on the validation set
y_val_pred = svm_model.predict(X_val)
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred)}")

print(classification_report(y_val, y_val_pred, target_names=unique_labels))


Validation Accuracy: 0.9902912621359223
                                      precision    recall  f1-score   support

              human.pedestrian.adult       0.99      1.00      0.99       978
              human.pedestrian.child       0.33      0.17      0.22         6
human.pedestrian.construction_worker       1.00      1.00      1.00        31
  human.pedestrian.personal_mobility       1.00      1.00      1.00         5
     human.pedestrian.police_officer       1.00      1.00      1.00         6
              movable_object.barrier       1.00      1.00      1.00       512
               movable_object.debris       1.00      1.00      1.00         1
    movable_object.pushable_pullable       0.92      1.00      0.96        11
          movable_object.trafficcone       1.00      1.00      1.00       248
          static_object.bicycle_rack       1.00      1.00      1.00        10
                     vehicle.bicycle       1.00      1.00      1.00        46
                   vehi

In [ ]:
# Make predictions on the test set
y_test_pred = svm_model.predict(X_test)

# Evaluate the model
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred)}")
print(classification_report(y_test, y_test_pred, target_names=unique_labels))


Test Accuracy: 0.9919093851132686
                                      precision    recall  f1-score   support

              human.pedestrian.adult       0.99      1.00      0.99       961
              human.pedestrian.child       0.67      0.40      0.50        10
human.pedestrian.construction_worker       1.00      1.00      1.00        31
  human.pedestrian.personal_mobility       1.00      1.00      1.00         4
     human.pedestrian.police_officer       1.00      1.00      1.00         2
              movable_object.barrier       1.00      1.00      1.00       445
               movable_object.debris       1.00      1.00      1.00         2
    movable_object.pushable_pullable       0.96      0.96      0.96        26
          movable_object.trafficcone       1.00      1.00      1.00       281
          static_object.bicycle_rack       1.00      1.00      1.00        11
                     vehicle.bicycle       1.00      1.00      1.00        54
                   vehicle.bu

In [ ]:
# Define the number of folds
k = 10
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=111)
# Perform k-fold cross-validation
scores = cross_val_score(svm_model, features, labels, cv=skf)
# Print the cross-validation scores
print(f"Cross-validation scores: {scores}")
print(f"Mean cross-validation score: {scores-mean}")

NameError: name 'svm_model' is not defined